In [49]:
from __future__ import print_function, division

In [50]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import scale
from sklearn.decomposition import KernelPCA

from sklearn.linear_model import Ridge, ElasticNet, BayesianRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from mlens.ensemble import SuperLearner
from sklearn.svm import SVR

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import r2_score

In [51]:
# update your data path
DATA_PATH = "/home/aunagar/Personal/Study/Sem1/Advanced ML/projects/task1/Task1/"

In [52]:
# import data
train_X = pd.read_csv(DATA_PATH + "X_train.csv")
train_Y = pd.read_csv(DATA_PATH + "y_train.csv")
test_X = pd.read_csv(DATA_PATH + "X_test.csv")
sample_submission = pd.read_csv(DATA_PATH + "sample.csv")

In [53]:
# split data
train_ids = train_X.iloc[:, 0]
train_features = train_X.iloc[:, 1:]
test_ids = test_X.iloc[:, 0]
test_features = test_X.iloc[:, 1:]

In [54]:
def missing_value_imputation(train, test):
    train = train.fillna(train.mean())
    
    test = test.fillna(train.mean())
    
    return train, test

def outlier_treatment(train, test):
    
    train_mean, train_std = train.mean(), train.std()
    
    train[train > train_mean + 2.*train_std] = np.nan
    train[train < train_mean - 2.*train_std] = np.nan
    
    test[test > train_mean + 2.*train_std] = np.nan
    test[test < train_mean - 2.*train_std] = np.nan
    
    train = train.fillna(train_mean)
    test = test.fillna(train_mean)
    
    return train, test

def normalize(train, test):
    train_mean, train_std = train.mean(), train.std()

    train = (train - train_mean)/train_std
    # test 
    test = (test - train_mean)/train_std
    
    return train, test

def remove_corr_features(train, test, threshold = 0.8):
    
    # Create correlation matrix
    corr_matrix = train.corr().abs()
    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    # Find index of feature columns with correlation greater than 0.7
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    # train
    train = train.drop(columns = to_drop)
    # test
    test = test.drop(columns = to_drop)
    
    return train, test
        
def preprocess(train, test):
    
    # impute missing values
    train, test = missing_value_imputation(train, test)
    
    # remove columns with 0 variance
    train_mean, train_std = train.mean(), train.std()
    train = train.iloc[:, np.where(train_std > 0.0)[0]]
    test = test.iloc[:, np.where(train_std > 0.0)[0]]
    
    # remove outliers
    train, test = outlier_treatment(train, test)
    
    # normalize
    train, test = normalize(train, test)
    
    # remove correlated features
    train, test = remove_corr_features(train, test)
    
    return train, test

In [55]:
train_features, test_features = preprocess(train_features, test_features)

In [56]:
###### Correlated feature removal #########
# Create correlation matrix
corr_matrix = train_features.corr().abs()
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Find index of feature columns with correlation greater than 0.7
to_drop = [column for column in upper.columns if any(upper[column] > 0.7)]

# train
train_features = train_features.drop(columns = to_drop)
# test
test_features = test_features.drop(columns = to_drop)

In [57]:
###### linear model
lr = ElasticNet(alpha = 0.5, l1_ratio=0.5)
validation_score = cross_val_score(lr, train_features, train_Y.iloc[:, 1:], cv = 5, scoring = 'r2')
print(validation_score.mean())

# train model on whole train data
lr.fit(X = train_features, y = train_Y.iloc[:, 1])

0.4817015978694041


ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [47]:
# finding to which features to remove
non_zero_weights = np.where(lr.coef_ != 0.)[0]

# removing these features from training data
train_features = train_features.iloc[:, non_zero_weights]
test_features = test_features.iloc[:, non_zero_weights]

In [48]:
#### training more complex model on this cleared date

###### linear model
alpha = np.arange(0.1, 2, 0.2)
l1_ratio = np.arange(0.1, 1, 0.2)
results = pd.DataFrame(columns=['alpha', 'l1_ratio', 'cv_score'])
for a in alpha:
    for l1 in l1_ratio:
        lr = ElasticNet(alpha = a, l1_ratio=l1)
        validation_score = cross_val_score(lr, train_features, train_Y.iloc[:, 1:], cv = 5, scoring = 'r2')
        results = results.append({'alpha':a, 'l1_ratio':l1, 'cv_score':validation_score.mean()}, ignore_index = True)

best_parameters = results.iloc[np.argmax(results.cv_score.values),:]
print(best_parameters['cv_score'])

# linear Elastic NEt
lr = ElasticNet(alpha = best_parameters['alpha'], l1_ratio = best_parameters['l1_ratio'])

# Gradient Boosting Regressor
gbr = GradientBoostingRegressor(loss='huber', learning_rate=0.05, n_estimators=1000, subsample=0.9,
                                criterion='friedman_mse', min_samples_split=10, min_samples_leaf=10,
                                min_weight_fraction_leaf=0.0, max_depth=7, min_impurity_decrease=0.0,
                                min_impurity_split=None, init=None, random_state=42, max_features=None,
                                alpha=0.9, verbose=2, max_leaf_nodes=None, warm_start=False, presort='auto',
                                validation_fraction=0.33, n_iter_no_change=None, tol=0.0001)

gbr_cross_val = cross_val_score(gbr, train_features, train_Y.iloc[:, 1], cv = 5, scoring = 'r2')
print(gbr_cross_val.mean())

# support vector regression
C = np.arange(1, 10, 1)
svr_params = pd.DataFrame(columns = ['C', 'cv_score'])
for c in C:
    svr = SVR(kernel = 'rbf', degree = 10, gamma = 'scale', C = c)
    cv_score = cross_val_score(svr, train_features, train_Y.iloc[:, 1], cv= 5, scoring = 'r2').mean()
    svr_params = svr_params.append({'C':c, 'cv_score':cv_score}, ignore_index = True)

best_params = svr_params.iloc[np.argmax(svr_params.cv_score.values), :]
print(best_params['cv_score'].mean())

svr = SVR(kernel= 'rbf', gamma = 'scale', C = best_params['C'])

0.582770370350672
      Iter       Train Loss      OOB Improve   Remaining Time 
         1          44.7100           1.2775            3.72m
         2          40.3126           2.4686            3.22m
         3          37.9108           1.9871            3.11m
         4          35.7126           1.4301            3.03m
         5          32.4681           1.4775            3.10m
         6          31.2209           0.9250            3.12m
         7          28.8169           1.5739            3.27m
         8          26.5536           1.3571            3.21m
         9          24.4394           1.5660            3.26m
        10          24.0506           0.8909            3.19m
        11          21.9853           1.2191            3.16m
        12          21.4879           0.7219            3.17m
        13          19.6538           0.8292            3.15m
        14          18.6114           0.6904            3.11m
        15          17.1176           0.8135       

       132           0.4906          -0.0014            2.49m
       133           0.4951          -0.0009            2.48m
       134           0.4528          -0.0039            2.48m
       135           0.4757           0.0023            2.48m
       136           0.4360           0.0101            2.48m
       137           0.4270           0.0023            2.48m
       138           0.4130           0.0060            2.47m
       139           0.4529          -0.0029            2.46m
       140           0.4241           0.0038            2.46m
       141           0.4198           0.0033            2.45m
       142           0.3775           0.0067            2.45m
       143           0.3964          -0.0024            2.44m
       144           0.3828           0.0001            2.44m
       145           0.3873           0.0137            2.43m
       146           0.4018          -0.0007            2.42m
       147           0.3925           0.0013            2.42m
       1

       266           0.0913          -0.0001            1.96m
       267           0.0936           0.0001            1.96m
       268           0.0827           0.0002            1.95m
       269           0.0880           0.0005            1.95m
       270           0.0876           0.0001            1.95m
       271           0.0834           0.0010            1.94m
       272           0.0781          -0.0037            1.94m
       273           0.0866           0.0005            1.94m
       274           0.0811           0.0002            1.93m
       275           0.0852           0.0006            1.93m
       276           0.0850          -0.0002            1.93m
       277           0.0759           0.0012            1.92m
       278           0.0803           0.0003            1.92m
       279           0.0691          -0.0009            1.91m
       280           0.0768          -0.0000            1.91m
       281           0.0883          -0.0002            1.90m
       2

       400           0.0304           0.0001            1.53m
       401           0.0265           0.0000            1.53m
       402           0.0289           0.0004            1.53m
       403           0.0291           0.0001            1.52m
       404           0.0254           0.0010            1.52m
       405           0.0256           0.0003            1.52m
       406           0.0228           0.0001            1.51m
       407           0.0273           0.0001            1.51m
       408           0.0259           0.0000            1.51m
       409           0.0264           0.0003            1.51m
       410           0.0263           0.0006            1.50m
       411           0.0248           0.0004            1.50m
       412           0.0263          -0.0001            1.50m
       413           0.0242           0.0015            1.49m
       414           0.0242          -0.0000            1.49m
       415           0.0261           0.0000            1.48m
       4

       533           0.0180           0.0003            1.10m
       534           0.0178          -0.0000            1.10m
       535           0.0156           0.0009            1.10m
       536           0.0151           0.0013            1.10m
       537           0.0163          -0.0000            1.09m
       538           0.0162           0.0001            1.09m
       539           0.0156           0.0000            1.09m
       540           0.0133           0.0003            1.08m
       541           0.0161           0.0000            1.08m
       542           0.0149           0.0003            1.08m
       543           0.0146          -0.0012            1.08m
       544           0.0138           0.0000            1.07m
       545           0.0156           0.0002            1.07m
       546           0.0149           0.0001            1.07m
       547           0.0155           0.0000            1.06m
       548           0.0261          -0.0066            1.06m
       5

...

       406           0.0269           0.0000            1.26m
       407           0.0272          -0.0000            1.26m
       408           0.0282           0.0002            1.26m
       409           0.0270          -0.0000            1.26m
       410           0.0264           0.0005            1.25m
       411           0.0266           0.0001            1.25m
       412           0.0221           0.0006            1.25m
       413           0.0251           0.0000            1.25m
       414           0.0250           0.0000            1.25m
       415           0.0216           0.0008            1.24m
       416           0.0235          -0.0002            1.24m
       417           0.0255          -0.0000            1.24m
       418           0.0235           0.0004            1.24m
       419           0.0224           0.0000            1.23m
       420           0.0239          -0.0000            1.23m
       421           0.0234           0.0001            1.23m
       4

       540           0.0100           0.0004           58.17s
       541           0.0109           0.0001           58.03s
       542           0.0090           0.0003           57.93s
       543           0.0099           0.0001           57.85s
       544           0.0098           0.0002           57.72s
       545           0.0102           0.0000           57.60s
       546           0.0099           0.0002           57.47s
       547           0.0109          -0.0005           57.34s
       548           0.0114           0.0000           57.20s
       549           0.0113          -0.0000           57.06s
       550           0.0107           0.0002           56.94s
       551           0.0093           0.0006           56.82s
       552           0.0129          -0.0014           56.66s
       553           0.0114          -0.0000           56.53s
       554           0.0125          -0.0000           56.39s
       555           0.0165          -0.0034           56.25s
       5

       674           0.0106           0.0000           40.51s
       675           0.0094          -0.0000           40.37s
       676           0.0097           0.0000           40.25s
       677           0.0099           0.0001           40.12s
       678           0.0092           0.0003           39.99s
       679           0.0087          -0.0000           39.87s
       680           0.0081           0.0007           39.75s
       681           0.0097           0.0000           39.62s
       682           0.0091           0.0000           39.48s
       683           0.0088          -0.0000           39.36s
       684           0.0088          -0.0001           39.23s
       685           0.0084           0.0001           39.10s
       686           0.0091           0.0000           38.97s
       687           0.0088           0.0001           38.84s
       688           0.0094           0.0000           38.72s
       689           0.0086           0.0001           38.59s
       6

       807           0.0062          -0.0000           23.68s
       808           0.0064           0.0000           23.55s
       809           0.0065           0.0000           23.42s
       810           0.0062           0.0000           23.30s
       811           0.0053           0.0001           23.18s
       812           0.0060           0.0001           23.06s
       813           0.0058           0.0001           22.95s
       814           0.0061           0.0000           22.83s
       815           0.0059           0.0000           22.70s
       816           0.0062          -0.0002           22.58s
       817           0.0058          -0.0000           22.46s
       818           0.0063           0.0000           22.33s
       819           0.0056           0.0002           22.21s
       820           0.0056          -0.0000           22.09s
       821           0.0061          -0.0000           21.96s
       822           0.0058          -0.0000           21.84s
       8

       940           0.0063           0.0003            7.32s
       941           0.0071          -0.0000            7.20s
       942           0.0065           0.0002            7.08s
       943           0.0067           0.0000            6.95s
       944           0.0062           0.0001            6.83s
       945           0.0054           0.0000            6.71s
       946           0.0056           0.0000            6.58s
       947           0.0063          -0.0000            6.46s
       948           0.0069          -0.0003            6.34s
       949           0.0068           0.0000            6.22s
       950           0.0072          -0.0000            6.09s
       951           0.0069           0.0000            5.97s
       952           0.0067           0.0000            5.85s
       953           0.0064           0.0000            5.72s
       954           0.0066           0.0000            5.60s
       955           0.0048           0.0004            5.48s
       9

In [29]:
class predictor:
    
    def __init__(self, predictor1, predictor2):
        self.pr1 = predictor1
        self.pr2 = predictor2
        
    def fit(self, X, Y):
        self.pr1.fit(X, Y)
        
        error = self.pr1.predict(X) - Y
        
        self.pr2.fit(X, error)
    
    def get_params(self):
        pass
    
    def predict(X):
        
        pr_1 = self.pr1.predict(X)
        pr_2 = self.pr2.predict(X)
        
        return pr1 - pr2

In [137]:
train_x, test_x, train_y, test_y = train_test_split(train_features, train_Y.iloc[:, 1], test_size = 0.33,
                                                    random_state = 42)

In [138]:
lr.fit(train_x, train_y)
svr.fit(train_x, train_y)
gbr.fit(train_x, train_y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.05, loss='huber', max_depth=7,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=10, min_samples_split=10,
                          min_weight_fraction_leaf=0.0, n_estimators=1000,
                          n_iter_no_change=None, presort='auto',
                          random_state=42, subsample=0.9, tol=0.0001,
                          validation_fraction=0.33, verbose=0,
                          warm_start=False)

In [139]:
lr_pred = lr.predict(test_x)
svr_pred = svr.predict(test_x)
gbr_pred = gbr.predict(test_x)

In [140]:
lr_scorelr.score(test_x, test_y)
svr_score = svr.score(test_x, test_y))
print(gbr.score(test_x, test_y))

0.571087397627106
0.5596892204391134
0.510028565950948


In [141]:
overall_pred = (lr_pred + svr_pred + gbr_pred)/3.

In [142]:
print(r2_score(test_y, overall_pred))

0.5710096591903691


### Predictions

In [97]:
best_model = None

In [99]:
lr_test_pred = lr.predict(test_features)
svr_test_pred = svr.predict(test_features)
gbr_test_pred = gbr.predict(test_features)

In [100]:
predicted = (lr_test_pred + svr_test_pred + gbr_test_pred)/3.

In [101]:
#predicted = best_model.predict(test_features)
sample_submission['y'] = predicted

In [102]:
sample_submission.to_csv("submissions/Ajay_13th_sub.csv", index = False)